In [39]:
#! /usr/bin/python3

import sys
import pennylane as qml
import numpy as np
import sklearn as skl
import autograd.numpy as np

def classify_data(X_train,Y_train,X_test,Y_test,**kwargs):
    """Develop and train your very own variational quantum classifier.

    Use the provided training data to train your classifier. The code you write
    for this challenge should be completely contained within this function
    between the # QHACK # comment markers. The number of qubits, choice of
    variational ansatz, cost function, and optimization method are all to be
    developed by you in this function.

    Args:
        X_train (np.ndarray): An array of floats of size (250, 3) to be used as training data.
        Y_train (np.ndarray): An array of size (250,) which are the categorical labels
            associated to the training data. The categories are labeled by -1, 0, and 1.
        X_test (np.ndarray): An array of floats of (50, 3) to serve as testing data.

    Returns:
        str: The predicted categories of X_test, converted from a list of ints to a
            comma-separated string.
    """

    # Use this array to make a prediction for the labels of the data in X_test
    predictions = []

    # QHACK #

    from autograd.numpy import exp,tanh

    # start by renormalizing training data to fit within [-1,1]
    """
    X_train = np.multiply(1.0,np.subtract(np.multiply(\
            np.divide(np.subtract(X_train,X_train.min()),\
                      (X_train.max()-X_train.min())),2.0),1.0))

    # start by renormalizing testing data to fit within [-1,1]
    X_test = np.multiply(1.0,np.subtract(np.multiply(\
            np.divide(np.subtract(X_test,X_test.min()),\
                      (X_test.max()-X_test.min())),2.0),1.0))
    """
    def statepreparation(a):
        qml.templates.embeddings.AngleEmbedding(a, wires=range(3), rotation='Y')

    def layer(W):
        qml.templates.layers.BasicEntanglerLayers(W, wires=range(3), rotation=qml.ops.RY)

    def hinge_loss(labels, predictions,type='L2'):
        loss = 0
        for l, p in zip(labels, predictions):
            if type=='L1':
                loss = loss + np.abs(l - p) # L1 loss
            elif type=='L2':
                loss = loss + (l - p) ** 2 # L2 loss
        loss = loss/len(labels)
        return loss

    def accuracy(labels, predictions):

        loss = 0
        tol = 0.05
        #tol = 0.1
        for l, p in zip(labels, predictions):
            if abs(l - p) < tol:
                loss = loss + 1
        loss = loss / len(labels)

        return loss

    def cost_fcn(params,circuit=None,ang_array=[], actual=[]):
        '''
        use MAE to start
        '''
        labels = {2:-1,1:1,0:0}
        w = params[-3:]
        theta = params[:-3]
        predictions = [2.*(1.0/(1.0+exp(np.dot(-w,circuit(theta, angles=x)))))- 1. for x in ang_array]
        return hinge_loss(actual, predictions)

    dev = qml.device("default.qubit", wires=3)
    @qml.qnode(dev)
    def small_circuit(params,angles=None):
        statepreparation(angles)
        W= np.reshape(params,(len(params)//3,3))
        layer(W)
        return qml.expval(qml.PauliZ(0)),qml.expval(qml.PauliZ(1)),qml.expval(qml.PauliZ(2))


    var = np.hstack((np.zeros(6),5*np.random.random(3)-2.5))
    steps = kwargs['s']
    batch_size = kwargs['batch_size']
    num_train = len(Y_train)
    validation_size = int(num_train//2)
    opt = qml.AdamOptimizer(kwargs['learning_rate'])

    for _ in range(steps):
        batch_index = np.random.randint(0, num_train, (batch_size,))
        X_train_batch = X_train[batch_index]
        Y_train_batch = Y_train[batch_index]

        var,cost = opt.step_and_cost(lambda v: cost_fcn(v, small_circuit,X_train_batch, Y_train_batch), var)

        if (_%3)==0:
            val_index = np.random.randint(0, num_train, (validation_size,))
            X_val_batch = X_train[val_index]
            Y_val_batch = Y_train[val_index]
            w = var[-3:]
            theta = var[:-3]
            pred = [np.round(2.*(1.0/(1.0+exp(np.dot(-w,small_circuit(theta, angles=x)))))- 1.,0) for x in X_val_batch]
            acc= accuracy(Y_val_batch,pred)
            if acc>0.98:
                break
    # need timing values from computing predictions
    predictions=[int(np.round(2.*(1.0/(1.0+exp(np.dot(-w,small_circuit(theta, angles=x)))))- 1.,0)) for x in X_test]
    err_rate = accuracy(predictions,Y_test)
    # QHACK #

    #return array_to_concatenated_string(predictions)
    return len(var),err_rate,'missing time'


def array_to_concatenated_string(array):
    """DO NOT MODIFY THIS FUNCTION.

    Turns an array of integers into a concatenated string of integers
    separated by commas. (Inverse of concatenated_string_to_array).
    """
    return ",".join(str(x) for x in array)


def concatenated_string_to_array(string):
    """DO NOT MODIFY THIS FUNCTION.

    Turns a concatenated string of integers separated by commas into
    an array of integers. (Inverse of array_to_concatenated_string).
    """
    return np.array([int(x) for x in string.split(",")])


def parse_input(giant_string):
    """DO NOT MODIFY THIS FUNCTION.

    Parse the input data into 3 arrays: the training data, training labels,
    and testing data.

    Dimensions of the input data are:
      - X_train: (250, 3)
      - Y_train: (250,)
      - X_test:  (50, 3)
    """
    X_train_part, Y_train_part, X_test_part = giant_string.split("XXX")

    X_train_row_strings = X_train_part.split("S")
    X_train_rows = [[float(x) for x in row.split(",")] for row in X_train_row_strings]
    X_train = np.array(X_train_rows)

    Y_train = concatenated_string_to_array(Y_train_part)

    X_test_row_strings = X_test_part.split("S")
    X_test_rows = [[float(x) for x in row.split(",")] for row in X_test_row_strings]
    X_test = np.array(X_test_rows)

    return X_train, Y_train, X_test

## Import data from sklearn

In [4]:
from sklearn import datasets

In [7]:
n_samples = 1500
noisy_circles = datasets.make_circles(n_samples=n_samples, factor=.5,
                                      noise=.05)
noisy_moons = datasets.make_moons(n_samples=n_samples, noise=.05)

In [13]:
X_train = noisy_circles[0][:1000]
Y_train = noisy_circles[1][:1000]
X_test = noisy_circles[0][500:]
Y_test = noisy_circles[1][500:]

In [42]:
batch_sets = [1,2,4,8,16]
learning_rates = [0.001,0.005,0.01,0.05,0.1]
hyperparameters = list(zip(batch_sets,learning_rates))
print(hyperparameters)
for idx,sdx in hyperparameters:
    print(classify_data(X_train,Y_train,X_test,Y_test,s=10,batch_size=sdx[0],learning_rate=sdx[1]))

[(1, 0.001), (2, 0.005), (4, 0.01), (8, 0.05), (16, 0.1)]


TypeError: 'float' object is not subscriptable

In [40]:
classify_data(X_train,Y_train,X_test,Y_test,s=10,batch_size=8,learning_rate=0.1)

(9, 0.495, 'missing time')